In [7]:
import cv2
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import time

In [ ]:
# Detection

def webcam_face_detection():
    # Load the pre-trained Haar Cascade face detector
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    
    # Initialize webcam (0 = default camera)
    cap = cv2.VideoCapture(0)
    
    # Check if webcam opened successfully
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    try:
        while True:
            # Read frame from webcam
            ret, frame = cap.read()
            if not ret:
                print("Error: Can't receive frame")
                break
            
            # Convert to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detect faces
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )
            # Draw rectangles around detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display the resulting frame
            cv2.imshow('Face Detection - Press Q to Quit', frame)
            
            # Break loop when 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the face detection
webcam_face_detection()

KeyboardInterrupt: 

In [9]:
import cv2
import deepface
import ipywidgets
print(cv2.__version__)
print(deepface.__version__)
print(ipywidgets.__version__)

4.11.0
0.0.93
8.0.4


In [10]:
import numpy as np
from deepface import DeepFace
import os
from datetime import datetime
import cv2

def detect_faces_and_attributes():
    # Create a directory to save images
    output_dir = "captured_faces"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    # Variables for attribute detection
    frame_skip = 10  # Process attributes every 10 frames
    frame_count = 0
    last_emotion = "None"
    last_emotion_score = 0.0
    last_age = "Unknown"
    last_gender = "Unknown"
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame")
                break
            
            # Convert to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detect faces
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )
            
            # Create a copy of the frame to annotate
            annotated_frame = frame.copy()
            
            # Process each face found
            for (x, y, w, h) in faces:
                # Draw rectangle around face
                color = (0, 255, 0)  # Green
                cv2.rectangle(annotated_frame, (x, y), (x+w, y+h), color, 2)
                
                # Attribute detection (emotion, age, gender) every frame_skip frames
                if frame_count % frame_skip == 0:
                    try:
                        # Extract face ROI
                        face_roi = frame[y:y+h, x:x+w]
                        if face_roi.size > 0:
                            # Analyze emotion, age, and gender
                            analysis = DeepFace.analyze(
                                img_path=face_roi,
                                actions=['emotion', 'age', 'gender'],
                                detector_backend='opencv',
                                enforce_detection=False
                            )
                            last_emotion = analysis[0]['dominant_emotion'].capitalize()
                            last_emotion_score = analysis[0]['emotion'][last_emotion.lower()]
                            last_age = analysis[0]['age']
                            last_gender = analysis[0]['dominant_gender']
                        else:
                            last_emotion = "No Face"
                            last_emotion_score = 0.0
                            last_age = "Unknown"
                            last_gender = "Unknown"
                    except Exception as e:
                        print(f"Attribute detection error: {e}")
                        last_emotion = "Error"
                        last_emotion_score = 0.0
                        last_age = "Unknown"
                        last_gender = "Unknown"
                
                # Display attributes
                emotion_text = f"{last_emotion}: {last_emotion_score:.1f}%"
                age_gender_text = f"Age: {last_age}, Gender: {last_gender}"
                cv2.putText(annotated_frame, emotion_text, (x, y-40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                cv2.putText(annotated_frame, age_gender_text, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                
                # Save the frame with detected face and text
                if frame_count % frame_skip == 0:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"{output_dir}/face_{timestamp}_{last_emotion}.jpg"
                    cv2.imwrite(filename, annotated_frame)
                    print(f"Saved image: {filename}")
            
            # Display frame count and instructions
            cv2.putText(annotated_frame, f"Faces: {len(faces)}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, "Press Q to Quit", (10, annotated_frame.shape[0]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            # Show the frame
            cv2.imshow('Face & Attribute Detection', annotated_frame)
            
            # Exit on 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            frame_count += 1
    
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Run the detection
if __name__ == "__main__":
    try:
        detect_faces_and_attributes()
    except ImportError as e:
        print(f"Import error: {e}")
        print("Installing required packages...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python", "deepface", "numpy", "--user"])
        print("Packages installed. Please run the program again.")

Action: gender: 100%|██████████| 3/3 [00:07<00:00,  2.47s/it] 


Saved image: captured_faces/face_20250507_182316_Sad.jpg


Action: gender: 100%|██████████| 3/3 [00:02<00:00,  1.16it/s]


Saved image: captured_faces/face_20250507_182319_Neutral.jpg


Action: gender: 100%|██████████| 3/3 [00:01<00:00,  1.86it/s]


...

Saved image: captured_faces/face_20250507_182349_Neutral.jpg


Action: gender: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


Saved image: captured_faces/face_20250507_182351_Sad.jpg


Action: gender: 100%|██████████| 3/3 [00:01<00:00,  2.13it/s]


Saved image: captured_faces/face_20250507_182353_Happy.jpg
